# JPL Small-Body Mission Design API Tutorial

This Jupyter Notebook introduces the use of the JPL Small-Body Mission Design API. It will demonstrate how to query small body data, extract trajectory information, and use this data for planning or analysis purposes.

In [24]:
# Section 1: Import Required Libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from astropy.time import Time
import numpy as np

In [25]:
# Section 2: API Overview and Base URL
base_url = "https://ssd-api.jpl.nasa.gov/mdesign.api"

In [26]:
# Section 3: Define a helper function to pretty-print JSON responses
def query_api(params):
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    return response.json()

In [36]:
# Section 4: Define Parameters for Pork-Chop Plot
# We'll sweep launch dates and TOF values for 1P/Halley
launch_start = Time('2025-01-01T00:00:00',format='isot').mjd

launch_span = 365 #Days 
tof_min = 180 #Days
tof_max = 720 #Days


In [38]:

# Section 5: Query and Build Pork-Chop Data Grid
target = "46P" #this is the short designation for the target
porkchop_data = []

print("Gathering pork-chop data (this may take a few minutes)...")
params = {
            "des": target,
            "mjd0": int(launch_date),
            "tof-min": tof_min,
            "tof-max": tof_max,
            "span": 90,
            "step":5,
        }
        
        try:
            result = query_api(params)
            if result["selectedMissions"]:
                best = result[0]
                porkchop_data.append({
                    "launch": launch_date,
                    "tof": best['MJDf']-best['MJD0'],
                    "vinf_dep": best['vinf_dep'],
                    "vinf_arr": best['vinf_arr'],
                    "c3": best['c3'],
                    "mass_dry": best.get("mass_dry"),
                    "mass_wet": best.get("mass_wet")
                })
        except Exception as e:
            print(f"Failed for {launch_date} TOF {tof}: {e}")    


Gathering pork-chop data (this may take a few minutes)...
Failed for 60765.0 TOF 180: 0
Failed for 60765.0 TOF 210: 0
Failed for 60765.0 TOF 240: 0
Failed for 60765.0 TOF 270: 0
Failed for 60765.0 TOF 300: 0
Failed for 60765.0 TOF 330: 0
Failed for 60765.0 TOF 360: 0
Failed for 60765.0 TOF 390: 0
Failed for 60765.0 TOF 420: 0
Failed for 60765.0 TOF 450: 0
Failed for 60765.0 TOF 480: 0
Failed for 60765.0 TOF 510: 0
Failed for 60765.0 TOF 540: 0
Failed for 60765.0 TOF 570: 0
Failed for 60765.0 TOF 600: 0
Failed for 60765.0 TOF 630: 0
Failed for 60765.0 TOF 660: 0
Failed for 60765.0 TOF 690: 0


In [ ]:
#Section 6: Build DataFrame and Generate Pork-Chop Plot
df = pd.DataFrame(porkchop_data)
df['launch_date'] = pd.to_datetime(df['launch'])
df['launch_jd'] = df['launch_date'].map(lambda d: d.toordinal())

# Pivot data for contour plot
pivot = df.pivot(index='tof', columns='launch_jd', values='c3')
X, Y = np.meshgrid(pivot.columns, pivot.index)
Z = pivot.values

# Convert launch dates back to datetime for labels
launch_labels = [datetime.fromordinal(int(jd)) for jd in pivot.columns]

plt.figure(figsize=(12, 7))
cp = plt.contourf(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(cp, label='Launch C3 (km²/s²)')
plt.title(f"Pork-Chop Plot for Mission to {target}")
plt.xlabel("Launch Date")
plt.ylabel("Time of Flight (days)")
plt.xticks(ticks=pivot.columns[::5], labels=[d.strftime('%Y-%m-%d') for d in launch_labels[::5]], rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

# Section 6: Mass Delivery Visualization
plt.figure(figsize=(12, 6))
filtered = df.dropna(subset=['mass_dry'])
sc = plt.scatter(filtered['launch_date'], filtered['tof'], c=filtered['mass_dry'], cmap='plasma', s=60)
plt.colorbar(sc, label='Dry Mass Delivered (kg)')
plt.xlabel("Launch Date")
plt.ylabel("Time of Flight (days)")
plt.title(f"Dry Mass Delivery Capability to {target}")
plt.grid(True)
plt.tight_layout()
plt.show()

_End of notebook_